In [ ]:
# testing uncentered-trained RNN 
# importing pytorch
import torch
print(torch.__version__)


In [ ]:
# running data config file
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate/model_boilerplate_remote.py'

In [ ]:
# specifying testing data
net = pd.read_csv('/home/christianl/Zhang-Lab/Zhang Lab Data/Full data files/network(full).tsv', sep = '\t')
x_in = x_test
y_out = y_test

In [ ]:
"""
Quick Diagnostic: What's Actually in Your state_dict?
======================================================
Run this to see the exact structure of your saved model.
"""

import torch

# Load your state_dict
state_dict_path = '/home/christian/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt'

print("=" * 70)
print("LOADING STATE_DICT")
print("=" * 70)

state_dict = torch.load(state_dict_path, map_location='cpu')

print(f"\nType: {type(state_dict)}")
print(f"\nIs it a dict? {isinstance(state_dict, dict)}")

# Check if it's actually the full model or just state_dict
if hasattr(state_dict, 'state_dict'):
    print("\n⚠️  This is a full model object, not just state_dict!")
    print("Extracting state_dict...")
    actual_state_dict = state_dict.state_dict()
else:
    actual_state_dict = state_dict

print("\n" + "=" * 70)
print("KEYS IN YOUR STATE_DICT")
print("=" * 70)

if isinstance(actual_state_dict, dict):
    print(f"\nTotal keys: {len(actual_state_dict.keys())}")
    print("\nAll keys:")
    for i, key in enumerate(actual_state_dict.keys(), 1):
        print(f"  {i}. {key}")
        print(f"     Shape: {actual_state_dict[key].shape if hasattr(actual_state_dict[key], 'shape') else 'N/A'}")
else:
    print(f"\n⚠️  Not a dict! Type: {type(actual_state_dict)}")
    print("\nTrying to access as object...")
    print(f"Attributes: {dir(actual_state_dict)}")

print("\n" + "=" * 70)
print("SEARCHING FOR LAYER WEIGHTS")
print("=" * 70)

# Search for different possible key patterns
patterns_to_check = [
    'input_layer.weights',
    'input_layer.weight',
    'signaling_network.weights',
    'signaling_network.weight',
    'output_layer.weights',
    'output_layer.weight',
    'project_input',
    'bionet',
    'project_output',
]

print("\nChecking for common patterns:")
for pattern in patterns_to_check:
    found = any(pattern in key for key in actual_state_dict.keys()) if isinstance(actual_state_dict, dict) else False
    status = "✓ FOUND" if found else "✗ Not found"
    print(f"  {status}: '{pattern}'")
    if found:
        matching_keys = [k for k in actual_state_dict.keys() if pattern in k]
        for key in matching_keys:
            print(f"      → {key}")

print("\n" + "=" * 70)
print("DIAGNOSIS")
print("=" * 70)

if isinstance(actual_state_dict, dict) and len(actual_state_dict) > 0:
    # Try to infer structure from keys
    all_keys = list(actual_state_dict.keys())
    
    print("\nYour state_dict structure:")
    
    # Check if keys have module prefix
    has_module_prefix = any('module.' in key for key in all_keys)
    if has_module_prefix:
        print("  ⚠️  Keys have 'module.' prefix (DataParallel model)")
    
    # Check key patterns
    if any('weight' in key for key in all_keys):
        print("  ✓ Has 'weight' keys")
    if any('bias' in key for key in all_keys):
        print("  ✓ Has 'bias' keys")
    if any('input' in key.lower() for key in all_keys):
        print("  ✓ Has input layer keys")
    if any('output' in key.lower() for key in all_keys):
        print("  ✓ Has output layer keys")
    if any('network' in key.lower() or 'bionet' in key.lower() for key in all_keys):
        print("  ✓ Has network/bionet layer keys")
    
    print("\n" + "=" * 70)
    print("NEXT STEPS")
    print("=" * 70)
    print("""
    Based on what we found, you may need to:
    
    1. If this is a full model object (not state_dict):
       → Use: model.state_dict() to get the actual state_dict
    
    2. If keys have different names:
       → I'll create an adapted loader for your specific structure
    
    3. If saved with DataParallel ('module.' prefix):
       → Need to strip the prefix
    
    Copy the output above and I'll help you adapt the scripts!
    """)

else:
    print("""
    ⚠️  Could not parse state_dict!
    
    This might be:
    1. A pickled full model (not state_dict)
    2. A different save format
    3. Corrupted file
    
    Try loading with:
    >>> model = torch.load(state_dict_path)
    >>> print(type(model))
    >>> print(dir(model))
    """)

In [7]:
analyze_what_you_have('/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt')

STATE_DICT ANALYSIS


/tmp/ipykernel_2640175/2818158888.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path, map_location='cpu')


KeyError: 'input_layer.weights'

In [ ]:
# initialization parameters
projection_amplitude_in = 1  # default of 1 
projection_amplitude_out = ???  # equal to the number of nodes in the final fixed network 
activation_function = 'MML'
bionet_params = {
    'target_steps': 100,
    'max_steps': 300,      # RNN iterations
    'exp_factor': 20,
    'leak': 0.01,          # for activation function
    'tolerance': 1e-5,     # convergence threshold
    'spectral_target': 1e-7
}
ban_list = None

dtype = torch.dtype=torch.float32
device = 'cpu' # was trained on gpu 
seed = 888 # default of 888 by Kejun's self.setseeds function unless specifcied




In [ ]:
# loading in uncentered data trained model learned weighted
uncentered_data_RNN_weights = torch.load('/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt')

In [ ]:
print("=== Model Parameters ===")
print(f"Input shape: {trained_model.X_in.shape}")
print(f"Output shape: {trained_model.y_out.shape}")
print(f"Number of nodes: {len(trained_model.node_idx_map)}")
print(f"Projection amplitude in: {trained_model.input_layer.projection_amplitude}")
print(f"Projection amplitude out: {trained_model.projection_amplitude_out}")
print(f"Device: {trained_model.device}")
print(f"Seed: {trained_model.seed}")
print(f"\nBioNet params: {trained_model.signaling_network.training_params}")

In [ ]:
# getting predicted values y_pred from test set 
rnn_y_pred = loaded_model.predict(x_test) 
print(type(rnn_y_pred), rnn_y_pred.shape)